In [1]:
import pandas as pd
import numpy as np

In [182]:
import vars

######################### CRIA df_rat
df_rat = pd.read_sql_table('tbl_rat', 'sqlite:///gdo.db')

df_rat = df_rat[
    (df_rat['NAT.CODIGO'].str.contains('Y0700[1345]')) |
    (df_rat['NAT.CODIGO'] == 'Y04012')  
]

df_rat.set_index('RAT.NUM_ATIVIDADE', inplace=True)

######################### BUSCA DADOS DE VIATURAS E ACRESCENTA EM df_rat
df_ratv = pd.read_sql_table('tbl_rat_veiculo', 'sqlite:///gdo.db')
df_ratv = df_ratv['NUM_ATIVIDADE'].value_counts()
df_ratv.rename('VIATURAS', inplace=True)
df_rat = df_rat.join(df_ratv, how='left')
del df_ratv
df_rat['VIATURAS'].fillna(0, inplace=True)
df_rat['VIATURAS'] = df_rat['VIATURAS'].astype(int)

######################### BUSCA DADOS DE EFETIVO E ACRESCENTA EM df_rat
df_rate = pd.read_sql_table('tbl_rat_efetivo', 'sqlite:///gdo.db')
df_rate = df_rate['NUM_ATIVIDADE'].value_counts()
df_rate.rename('EFETIVO', inplace=True)
df_rat = df_rat.join(df_rate, how='left')
del df_rate
df_rat['EFETIVO'].fillna(0, inplace=True)
df_rat['EFETIVO'] = df_rat['EFETIVO'].astype(int)

######################### BUSCA DADOS DE PRODUTIVIDADE E ACRESCENTA EM df_rat
df_ratp = pd.read_sql_table('tbl_rat_produtividade', 'sqlite:///gdo.db')
df_ratp_efet = df_ratp[
    df_ratp['DESCRICAO'].isin(vars.itens_efet)
].drop_duplicates('RAT.NUM_ATIVIDADE').set_index('RAT.NUM_ATIVIDADE')['QUANTIDADE'].map(lambda qtd:1).rename('EFETIVIDADE_PARCIAL')
df_rat = df_rat.join(df_ratp_efet, how='left')
del df_ratp_efet
df_rat['EFETIVIDADE_PARCIAL'].fillna(0, inplace=True)
df_rat['EFETIVIDADE_PARCIAL'] = df_rat['EFETIVIDADE_PARCIAL'].astype(int)

df_itens_ee = df_ratp[
    df_ratp['DESCRICAO'].isin([
        'Qde de veiculos fiscalizados',
        'Qde de pessoas abordadas',
        'Qde de locais fiscalizados',
        'Qde de pessoas que sopraram o etilometro'
    ])
]
df_itens_ee = pd.pivot_table(df_itens_ee, columns='DESCRICAO', index='RAT.NUM_ATIVIDADE')
df_itens_ee.columns = df_itens_ee.columns.droplevel()
df_rat = df_rat.join(df_itens_ee, how='left')
for i in [
    'Qde de veiculos fiscalizados',
    'Qde de pessoas abordadas',
    'Qde de locais fiscalizados',
    'Qde de pessoas que sopraram o etilometro'
]:
    df_rat[i].fillna(0, inplace=True)
    df_rat[i] = df_rat[i].astype('int32')
del df_ratp

cond71 = (
    # Y07001 - OPERACAO DE BATIDA POLICIAL
    (
        (df_rat['NAT.CODIGO'] == 'Y07001') &
        (df_rat['TEMPO_INT'] >= 30) &
        (df_rat['EFETIVO'] >= 2)
    ) &
    (
        (df_rat['Qde de pessoas abordadas'] >= 5) |
        (df_rat['Qde de veiculos fiscalizados'] >= 2)
    )
)
cond73 = (
    # Y07003 - OPERACAO DE INCURSAO EM ZONA QUENTE DE CRIMINALIDADE
    (
        (df_rat['NAT.CODIGO'] == 'Y07003') &
        (df_rat['TEMPO_INT'] >= 30) &
        (df_rat['EFETIVO'] >= 3) &
        (df_rat['VIATURAS'] >= 1)
    ) &
    (
        (df_rat['Qde de pessoas abordadas'] >= 5) |
        (df_rat['Qde de veiculos fiscalizados'] >= 2) |
        (df_rat['Qde de locais fiscalizados'] >= 2)
    )
)
cond74 = (
    # Y07004 - OPERACAO DE CERCO / BLOQUEIO / INTERCEPTACAO
    (
        (
            (df_rat['NAT.CODIGO'] == 'Y07004') &            
            (df_rat['EFETIVO'] >= 3) &
            (df_rat['VIATURAS'] >= 2)
        ) &
        (
            (df_rat['Qde de pessoas abordadas'] >= 1) |
            (df_rat['Qde de veiculos fiscalizados'] >= 1)
        )
    )
)
cond75 = (
    # Y07005 - OPERACAO DE OCUPACAO DE PTOS DE ZONA QUENTE DE CRIMINALIDADE
    (
        (df_rat['NAT.CODIGO'] == 'Y07005') &
        (df_rat['TEMPO_INT'] >= 60) &
        (df_rat['EFETIVO'] >= 3) &
        (df_rat['VIATURAS'] >= 1)
    ) &
    (
        (df_rat['Qde de pessoas abordadas'] >= 5) |
        (df_rat['Qde de veiculos fiscalizados'] >= 2) |
        (df_rat['Qde de locais fiscalizados'] >= 2)
    )    
)
cond412 = (
    # Y04012 - OPERACAO LEI SECA
    (
        (df_rat['NAT.CODIGO'] == 'Y04012') &
        (df_rat['TEMPO_INT'] >= 30) &
        (df_rat['EFETIVO'] >= 2) &
        (df_rat['VIATURAS'] >= 1)
    ) &
    (
        (df_rat['Qde de pessoas abordadas'] >= 3) |
        (df_rat['Qde de pessoas que sopraram o etilometro'] >= 3) |
        (df_rat['Qde de veiculos fiscalizados'] >= 3)
    )
)

cond_ee = [
    cond71 |
    cond73 |
    cond74 |
    cond75 |
    cond412
]

df_rat['EFICIENCIA_E_EFICACIA'] = np.select(cond_ee,[1],default=0)
df_rat['EFETIVIDADE'] = np.select(
    [(df_rat['EFETIVIDADE_PARCIAL'] == 1) & (df_rat['EFICIENCIA_E_EFICACIA'] == 1)],
    [1],
    default=0
)

df_rat.head()

,NAT.CODIGO,NAT.DESCRICAO,DTA_HRA_INCLUSAO,DTA_INICIO,HRA_INICIO,DTA_TERMINO,HRA_TERMINO,DES_ALVO_EVENTO,DES_LUGAR,NOM_OPERACAO,...,CIA,VIATURAS,EFETIVO,EFETIVIDADE_PARCIAL,Qde de locais fiscalizados,Qde de pessoas abordadas,Qde de pessoas que sopraram o etilometro,Qde de veiculos fiscalizados,EFICIENCIA_E_EFICACIA,EFETIVIDADE
RAT.NUM_ATIVIDADE,,,,,,,,,,,,,,,,,,,,,
2018-046472984-001,Y07001,OPERACAO DE BATIDA POLICIAL,17/10/2018,17/10/2018,18:43,17/10/2018,19:00,TRANSEUNTE,VIA DE ACESSO PUBLICA,,...,139 CIA,1,2,0,0,0,0,0,0,0
2018-051709149-001,Y07001,OPERACAO DE BATIDA POLICIAL,20/11/2018,20/11/2018,05:25,20/11/2018,07:15,TRANSEUNTE,METALURGICA / SIDERURGICA,,...,142 CIA,1,2,0,0,0,0,0,0,0
2018-052156427-001,Y07001,OPERACAO DE BATIDA POLICIAL,22/11/2018,22/11/2018,20:20,22/11/2018,20:45,TRANSEUNTE,VIA DE ACESSO PUBLICA,,...,142 CIA,1,2,0,0,0,0,0,0,0
2018-053752437-001,Y07001,OPERACAO DE BATIDA POLICIAL,03/12/2018,03/12/2018,07:38,03/12/2018,09:40,TRANSEUNTE,VIA DE ACESSO PUBLICA,,...,139 CIA,1,2,0,0,0,0,0,0,0
2018-054262980-001,Y07001,OPERACAO DE BATIDA POLICIAL,06/12/2018,06/12/2018,01:01,06/12/2018,02:50,MOTOCICLETA,VIA DE ACESSO PUBLICA,,...,139 CIA,1,2,0,0,0,0,0,0,0


In [169]:
df_rat_rqv_23 = df_rat[
    (df_rat['EFICIENCIA_E_EFICACIA'] == 1) &
    (df_rat['NAT.CODIGO'].str.contains('Y0700[1345]')) &
    ( ~ df_rat['NOM_UNID_RESPONSAVEL'].str.contains('IND PE')) &
    ( ~ df_rat['NOM_UNID_RESPONSAVEL'].str.contains('BPE')) &
    ( ~ df_rat['NOM_UNID_RESPONSAVEL'].str.contains('CPE')) &
    (df_rat['ANO'] == 2020)
            
].groupby(['ANO','MES','CIA','SETOR']).sum()[['EFICIENCIA_E_EFICACIA','EFETIVIDADE']]

m = df_rat_rqv_23['EFICIENCIA_E_EFICACIA'].unstack(level=['SETOR'])
m = m.assign(TOTAL = m.sum(1))
m = m.stack(level='SETOR')

m = m.unstack(level='MES')
m = m.assign(TOTAL = m.sum(1))
m.columns = pd.MultiIndex.from_product([['RQV'],['EFICIENCIA_E_EFICACIA'],list(m.columns)], names=['INDICADOR','TIPO','MES'])


m2 = df_rat_rqv_23['EFETIVIDADE'].unstack(level=['SETOR'])
m2 = m2.assign(TOTAL = m2.sum(1))
m2 = m2.stack(level='SETOR')

m2 = m2.unstack('MES')
m2 = m2.assign(TOTAL = m2.sum(1))
m2.columns = pd.MultiIndex.from_product([['RQV'],['EFETIVIDADE'],list(m2.columns)], names=['INDICADOR','TIPO','MES'])

m3 = pd.concat([m,m2],axis=1)
# m3.columns = ['EFICIENCIA_E_EFICACIA','EFETIVIDADE']
for col in m3.columns:
    m3[col] = m3[col].astype('int32')
    
m3

INDICADOR                                                          RQV       \
TIPO                                             EFICIENCIA_E_EFICACIA        
MES                                                                  1    2   
ANO  CIA     SETOR                                                            
2020 139 CIA CLAUDIO                                                77  113   
             PLANALTO                                               30   46   
             SAO JOSE                                              107   75   
             TOTAL                                                 214  234   
     142 CIA CARMO DO CAJURU/SAO GONCALO DO PARA                   150  180   
             NITEROI                                               136   98   
             PORTO VELHO                                           126   80   
             TOTAL                                                 412  358   
     51 CIA  CENTRO/PADRE EUSTAQUIO                                 78   45   
             LOURDES/ITATIAIUCU                                     86   54   
             SANTANENSE/MORADA NOVA                                 87   53   
             TOTAL                                                 251  152   
     53 CIA  ALTO GOIAS                                             25   15   
             BOM PASTOR                                            120   96   
             HIPER CENTRO                                           76   62   
             TOTAL                                                 221  173   

INDICADOR                                                                     \
TIPO                                                        EFETIVIDADE        
MES                                                 3 TOTAL           1    2   
ANO  CIA     SETOR                                                             
2020 139 CIA CLAUDIO                               98   288          15   14   
             PLANALTO                              33   109           4   10   
             SAO JOSE                              76   258          25   15   
             TOTAL                                207   655          44   39   
     142 CIA CARMO DO CAJURU/SAO GONCALO DO PARA  156   486          44   63   
             NITEROI                               93   327          43   37   
             PORTO VELHO                           80   286          55   41   
             TOTAL                                329  1099         142  141   
     51 CIA  CENTRO/PADRE EUSTAQUIO                40   163          36   18   
             LOURDES/ITATIAIUCU                    37   177          23   19   
             SANTANENSE/MORADA NOVA                42   182          33   21   
             TOTAL                                119   522          92   58   
     53 CIA  ALTO GOIAS                            19    59          15    7   
             BOM PASTOR                            84   300          32   18   
             HIPER CENTRO                          76   214           7    5   
             TOTAL                                179   573          54   30   

INDICADOR                                                    
TIPO                                                         
MES                                                 3 TOTAL  
ANO  CIA     SETOR                                           
2020 139 CIA CLAUDIO                               14    43  
             PLANALTO                               5    19  
             SAO JOSE                              17    57  
             TOTAL                                 36   119  
     142 CIA CARMO DO CAJURU/SAO GONCALO DO PARA   49   156  
             NITEROI                               42   122  
             PORTO VELHO                           46   142  
             TOTAL                                137   420  
     51 CIA  CENTRO/PADRE EUSTAQUIO                23    77  
            

In [329]:
df_ols = df_rat[
    (df_rat['NAT.CODIGO'] == 'Y04012') &
    ( ~ df_rat['NOM_UNID_RESPONSAVEL'].str.contains('PRV')) &
    (df_rat['ANO'] == 2020)
]
df_ols_resumo = df_ols.groupby(['ANO','MES','CIA']).sum()[['EFETIVIDADE']]

df_ols_resumo = df_ols_resumo.unstack(level='MES')
df_ols_resumo.fillna(0, inplace=True)
df_ols_resumo.columns = df_ols_resumo.columns.droplevel(0)
df_ols_resumo = df_ols_resumo.assign(ACUM = df_ols_resumo.sum(1))

df_ols_resumo.loc[(2020,'23 BPM'),:] = df_ols_resumo.sum()
df_ols_resumo = df_ols_resumo.astype('int32')

df_ols_resumo

MES           1  2  3  ACUM
ANO  CIA                   
2020 139 CIA  1  0  2     3
     142 CIA  0  1  0     1
     51 CIA   0  3  0     3
     53 CIA   2  1  1     4
     23 BPM   3  5  3    11

In [ ]:
cond_unid = (
    # Y0700X
    (
        (df_rat['NAT.CODIGO'].str.contains('Y0700[1345]')) &
        (
            ( ~ df_rat['NOM_UNID_RESPONSAVEL'].str.contains('IND PE')) |
            ( ~ df_rat['NOM_UNID_RESPONSAVEL'].str.contains('CPE'))
        )        
    ) |
    # Y04012
    (
        (df_rat['NAT.CODIGO'] == 'Y04012') &                    
        ( ~ df_rat['NOM_UNID_RESPONSAVEL'].str.contains('CPRV'))        
    )

)

In [82]:
df_rat[
    
        df_rat['EFETIVIDADE'] == 1
    
].groupby(['NOM_UNID_RESPONSAVEL']).count()

,NAT.CODIGO,NAT.DESCRICAO,DTA_HRA_INCLUSAO,DTA_INICIO,HRA_INICIO,DTA_TERMINO,HRA_TERMINO,DES_ALVO_EVENTO,DES_LUGAR,NOM_OPERACAO,...,CIA,VIATURAS,EFETIVO,EFETIVIDADE_PARCIAL,Qde de locais fiscalizados,Qde de pessoas abordadas,Qde de pessoas que sopraram o etilometro,Qde de veiculos fiscalizados,EFICIENCIA_E_EFICACIA,EFETIVIDADE
NOM_UNID_RESPONSAVEL,,,,,,,,,,,,,,,,,,,,,
1 GP/1 PEL MAMB/7 CIA PM MAMB/BPM MAMB,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
1 GP/3 PEL/139 CIA PM/23 BPM/7 RPM,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
1 GP/3 PEL/142 CIA PM/23 BPM/7 RPM,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
1 PEL/139 CIA PM/23 BPM/7 RPM,13,13,13,13,13,13,13,13,13,13,...,13,13,13,13,13,13,13,13,13,13
1 PEL/142 CIA PM/23 BPM/7 RPM,6,6,6,6,6,6,6,6,6,6,...,6,6,6,6,6,6,6,6,6,6
139 CIA PM/23 BPM/7 RPM,294,294,294,294,294,294,294,294,294,294,...,294,294,294,294,294,294,294,294,294,294
142 CIA PM/23 BPM/7 RPM,683,683,683,683,683,683,683,683,683,683,...,683,683,683,683,683,683,683,683,683,683
2 BPE/2 RPM,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2 GP/3 PEL/142 CIA PM/23 BPM/7 RPM,152,152,152,152,152,152,152,152,152,152,...,152,152,152,152,152,152,152,152,152,152
